# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Chinese restaurant** in **Toronto**, Canada.

Since there are lots of restaurants in Toronto we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no or few Chinese restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Chinese restaurants in the neighborhood, if any
* review of the Chinese restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use definition from [this wikipedia link](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* number of restaurants and their type, review and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Toronto neighborhood will be obtained using a csv file input